# Inference with Furiosa-LLM 

In [ ]:
from furiosa_llm import LLM, SamplingParams

# Load the Llama 3.1 8B Instruct model
llm = LLM.load_artifact("furiosa-ai/Llama-3.1-8B-Instruct-FP8", devices="npu:0")

# You can specify various parameters for text generation
sampling_params = SamplingParams(max_tokens=100, top_p=0.3, top_k=100)


Fetching 170 files: 100%|██████████| 170/170 [00:00<00:00, 5983.82it/s]
INFO:2025-06-04 02:17:38+0000 Prefill buckets with output size: [BucketWithOutputLogitsSize(bucket=Bucket(batch_size=1, attention_size=256, kv_cache_size=0),
                            output_logits_size=1),
 BucketWithOutputLogitsSize(bucket=Bucket(batch_size=1, attention_size=320, kv_cache_size=0),
                            output_logits_size=1),
 BucketWithOutputLogitsSize(bucket=Bucket(batch_size=1, attention_size=384, kv_cache_size=0),
                            output_logits_size=1),
 BucketWithOutputLogitsSize(bucket=Bucket(batch_size=1, attention_size=512, kv_cache_size=0),
                            output_logits_size=1),
 BucketWithOutputLogitsSize(bucket=Bucket(batch_size=1, attention_size=640, kv_cache_size=0),
                            output_logits_size=1),
 BucketWithOutputLogitsSize(bucket=Bucket(batch_size=1, attention_size=768, kv_cache_size=0),
                            output_logits_siz

2025-06-04T02:17:38.996881029Z  INFO furiosa_generator::backend::furiosa_rt: Trying to open:
DeviceRow([Device::npu_fused(0, 0..=7)])
2025-06-04T02:17:39.405244807Z  INFO furiosa_generator::backend: KV caches on for each layer (I8, total 8.4 MB * num_blocks over 32 layers)  will be allocated
2025-06-04T02:17:39.426210269Z  INFO furiosa_generator::backend::furiosa_rt: Loading 18955 parameters from storages has started ...
2025-06-04T02:17:39.426733339Z  INFO furiosa_sprinter::buffer::alloc: Support for huge page size of 2 MiB has been detected.
2025-06-04T02:17:55.30925103Z  INFO furiosa_generator::backend::furiosa_rt: 18955 parameters (12.8 GiB) has been successfully loaded (15 secs).
2025-06-04T02:17:55.709550566Z  INFO furiosa_generator::backend: Determine the maximized available num_block as 424338
2025-06-04T02:17:56.986897218Z  INFO furiosa_generator::scheduler::action_provider: Preparing Backend (DeviceIndex(0): npu:0):
2025-06-04T02:17:56.986922518Z  INFO furiosa_generator::sche

### 1. Single batch inference

In [4]:
# Prompt for the model
message = [{"role": "user", "content": "What is the capital of France?"}]
prompt = llm.tokenizer.apply_chat_template(message, tokenize=False)

# Generate text
response = llm.generate([prompt], sampling_params)

# Print the output of the model
print(response[0].outputs[0].text)


2025-06-04T02:18:47.904529276Z  INFO furiosa_generator::scheduler::hf_compat: num samples received: 1
assistant

The capital of France is Paris.


### 2. Multi batch inference 

In [29]:
messages = [[{"role": "user", "content": "What is the capital of France?"}],
            [{"role": "user", "content": "What is the capital of Germany?"}]]

prompts = [llm.tokenizer.apply_chat_template(message, tokenize=False) for message in messages]

# Generate text
responses = llm.generate(prompts, sampling_params)

# Print the output of the model
outputs = [responses[i].outputs[0].text.split("assistant\n\n")[-1] for i in range(len(responses))]
for i, output in enumerate(outputs):
    print(f"Batch {i+1}")
    print(f"Question {i + 1}: {messages[i][0]['content']}")
    print(f"Response {i + 1}: {output}")
    print("====================================================")

2025-06-04T02:33:22.691431827Z  INFO furiosa_generator::scheduler::hf_compat: num samples received: 2
Batch 1
Question 1: What is the capital of France?
Response 1: The capital of France is Paris.
Batch 2
Question 2: What is the capital of Germany?
Response 2: The capital of Germany is Berlin.


### 3. Async single batch inference

In [31]:
import asyncio
 
async def async_single_batch_inference():
    # Prompt for the model
    message = [{"role": "user", "content": "What is the capital of France?"}]
    prompt = llm.tokenizer.apply_chat_template(message, tokenize=False)

    # Generate text and print each token at a time
    async for output_txt in llm.stream_generate(prompt, sampling_params):
        print(output_txt, end="", flush=True)

await async_single_batch_inference()

assistant

The capital of France is Paris.